# Combine agents and vector stores
This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a RetrievalQA and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set return_direct=True to really just use the agent as a router.

In [ ]:
!pip install langchain openai chromadb langchain-community

In [ ]:
!pip install langchain-openai

In [ ]:
!pip install langchain-google-genai

In [ ]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
import os
import getpass

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [ ]:
doc_path = ""

In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(openai_api_key="sk-proj-c7BfnwNqZu4r43n9qCwKT3BlbkFJYvmLjhtpOtaxpJ95KMSN")
docsearch = Chroma.from_documents(texts, embeddings, collection_name="1")

In [ ]:
waterline = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [ ]:
from langchain_community.document_loaders import TextLoader

doc_path2= ""
loader2 = TextLoader(doc_path2)
documents2 = loader2.load()
text_splitter2 = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts2 = text_splitter2.split_documents(documents2)

embeddings2 = OpenAIEmbeddings(openai_api_key="sk-proj-c7BfnwNqZu4r43n9qCwKT3BlbkFJYvmLjhtpOtaxpJ95KMSN")
docsearch2 = Chroma.from_documents(texts2, embeddings2, collection_name="line2")

In [ ]:
waterline2 = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch2.as_retriever()
)

In [ ]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [ ]:
tools = [
    Tool(
        name="A QA system",
        func=1.run,
        description="useful for when you need to answer questions about the A.",
    ),
    Tool(
        name="QA System",
        func=2.run,
        description="useful for when you need to answer questions about the B",
    ),
]

In [ ]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent.run(
    ""
)

In [ ]:
agent.run(
    ""
)